In [1]:
#!pip install python-dotenv
#!pip install openai
#!pip install imageio
#!pip install azure-ai-formrecognizer
#!pip install azure-core

In [2]:
from openai import AzureOpenAI
import dotenv
import os
import base64
from azure.core.credentials import AzureKeyCredential

dotenv.load_dotenv(override=True)

True

In [3]:
client = AzureOpenAI(
    azure_endpoint=os.getenv('AzureOpenAiEndpoint'),
    api_key=os.getenv('AzureOpenAiKey'),
    azure_deployment='gpt-4o',
    api_version='2024-02-15-preview'
)

In [4]:
with open("graph1.png", "rb") as image_file:
    graph1 = base64.b64encode(image_file.read()).decode('utf-8')
with open("graph2.png", "rb") as image_file:
    graph2 = base64.b64encode(image_file.read()).decode('utf-8')
with open("graph3.png", "rb") as image_file:
    graph3 = base64.b64encode(image_file.read()).decode('utf-8')


In [10]:
def analyze_document_png(encoded_image,prompt):
    res = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/png;base64,{encoded_image}"}
                    },
                ],
            }
        ],
    )
    return res.choices[0].message.content

def analyze_document_jpg(encoded_image,prompt):
    res = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt },
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpg;base64,{encoded_image}"}
                    },
                ],
            }
        ],
    )
    return res.choices[0].message.content

In [6]:
prompt = 'Describe the given graph with as much as details as possible'
analyze_document_png(graph1, prompt)

'The graph presents data on the total number of prescriptions and the total number of patients each year from 2013 to 2021.\n\n1. **Total Prescriptions (blue line)**:\n   - **2013**: Approximately 19,000,000 prescriptions.\n   - **2014**: Slight increase to about 20,000,000 prescriptions.\n   - **2015**: Peak at approximately 22,000,000 prescriptions, the highest point on the graph.\n   - **2016**: Slight decrease to around 21,000,000 prescriptions.\n   - **2017**: Noticeable drop to around 19,000,000 prescriptions.\n   - **2018**: Marginal recovery to about 20,000,000 prescriptions.\n   - **2019**: Small drop again to about 19,500,000 prescriptions.\n   - **2020**: Stability continues around 19,000,000 prescriptions.\n   - **2021**: Slight increase back up to approximately 20,000,000 prescriptions.\n\n2. **Total Patients (red line)**:\n   - Throughout the period 2013 to 2021, the total number of patients remains relatively constant, fluctuating mildly around 10,000,000 patients. The t

In [13]:
prompt = 'Show the total precscriptions as well as the total patients for 2015 and 2017'
analyze_document_png(graph1, prompt)

'To determine the total prescriptions and total patients for 2015 and 2017, we can use the graph provided.\n\n**2015:**\n- Total Prescriptions: Approximately 22,000,000\n- Total Patients: Approximately 9,000,000\n\n**2017:**\n- Total Prescriptions: Approximately 20,000,000\n- Total Patients: Approximately 9,000,000\n\nThese values are estimated based on the data points in the graph.'

In [16]:
prompt = 'Describe the graph as best as you can and include as much as information as possibe to answer future questions about this graph based on the text you provide.'
graph_desc = analyze_document_png(graph1, prompt)

In [18]:
def document_ask(prompt,analyzedDocumentContent):
    response =client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {
                'role': 'user',
                'content': prompt
            },
            {
                'role': 'system',
                'content': f'You know about this file: {analyzedDocumentContent}'
            }
        ]
    )
    return response.choices[0].message.content

In [19]:
prompt = 'What was the total Prescriptions in 2016?'
document_ask(graph_desc, prompt)

'In 2016, the total number of prescriptions was approximately 21 million.'

In [12]:
prompt = 'Describe the given graphs with as much as details as possible'
system_message = "Average monthly rainfall for the period 2000-2018 (bar chart in blue, Tenkodogo station), air temperatures for the period 2000-2018 (red line with dots, Fada N'Gourma station) and satellite-derived surface water temperatures for the period 2000-2018 (blue line with crosses). Annual rainfall from 2000 to 2018 is shown in the inset (no data are available for 2001, 2003-2005 and 2010). https://doi.org/10.1371/journal.pntd.0009634.g002"
analyze_document_png(graph2, prompt+system_message)

"The image provides a comprehensive visualization of climate data, including average monthly rainfall, air temperatures, and satellite-derived surface water temperatures from 2000 to 2018. Here's a detailed description:\n\n### Main Graph\nThe main graph displays three distinct sets of data over the months of January to December:\n\n1. **Average Monthly Rainfall (Tenkodogo station)**\n    - Represented by blue bars.\n    - No significant rainfall occurs from January to April and in December.\n    - There is a noticeable increase starting in May, peaking in August (around 280 mm), and tapering off after September.\n    - The highest rainfall is observed in August, followed by July and September.\n\n2. **Air Temperatures (Fada N'Gourma station)**\n    - Represented by a red line with dots.\n    - There's a steady increase from January, peaking in April (around 30°C), then steadily decreasing until August.\n    - After August, the temperature slightly rises again, reaching another peak in 

In [8]:
prompt = 'Describe the given graph with as much as details as possible'
analyze_document_png(graph3, prompt)

"The graph provides detailed information on PM₂.₅ concentrations across India in the year 2000. PM₂.₅ refers to particulate matter less than 2.5 micrometers in diameter, which is a critical indicator of air pollution.\n\n### Map Details\nThe map of India illustrates the concentration of PM₂.₅ across various regions with the following color coding:\n- Dark green (0 to 5 µg/m³)\n- Light green (5 to 20 µg/m³)\n- Yellow-green (20 to 30 µg/m³)\n- Yellow (30 to 40 µg/m³)\n- Light orange (40 to 50 µg/m³)\n- Darker orange (50 to 60 µg/m³)\n- Light red (60 to 70 µg/m³)\n- Darker red (70 to 80 µg/m³)\n- Dark red (80 to 90 µg/m³)\n- Darkest red (90 to 100 µg/m³)\n\n### Population Exposure\nThe bar chart on the top right shows the percentage of India's population exposed to different pollution levels:\n- 25.8% of the population is exposed to PM₂.₅ concentrations between 20 to 30 µg/m³.\n- 19.7% of the population is exposed to concentrations between 30 to 40 µg/m³.\n- 14.7% of the population is exp

In [11]:
prompt = 'Describe the landscape in the given image and highlight the areas with huge polutions'
analyze_document_png(graph3, prompt)

"The provided image displays a map of India indicating the concentration of PM2.5 particles in the year 2000. This map is used to assess air quality by measuring the concentration of fine particulate matter (PM2.5) in various regions, which can affect respiratory health.\n\n### Landscape Analysis:\n- The map is divided into various regions with color-coded PM2.5 concentration levels:\n  - **Deep Green (0 to 5 µg/m³)**: Minimal concentration areas, indicating very low pollution levels.\n  - **Light Green (5 to 20 µg/m³)**: Low concentration areas.\n  - **Yellow (20 to 40 µg/m³)**: Moderate concentration areas.\n  - **Light Orange (40 to 60 µg/m³)**: High concentration areas.\n  - **Orange (60 to 80 µg/m³)**: Very high concentration areas.\n  - **Dark Orange/Red (80 to 100 µg/m³)**: Extremely high concentration areas.\n\n### Areas with High Pollution:\n- **Northern India (especially the Indo-Gangetic Plain)**: This area has the highest levels of PM2.5, with significant portions in the da

The provided image displays a map of India indicating the concentration of PM2.5 particles in the year 2000. This map is used to assess air quality by measuring the concentration of fine particulate matter (PM2.5) in various regions, which can affect respiratory health.\n\n### Landscape Analysis:\n- The map is divided into various regions with color-coded PM2.5 concentration levels:\n  - **Deep Green (0 to 5 µg/m³)**: Minimal concentration areas, indicating very low pollution levels.\n  - **Light Green (5 to 20 µg/m³)**: Low concentration areas.\n  - **Yellow (20 to 40 µg/m³)**: Moderate concentration areas.\n  - **Light Orange (40 to 60 µg/m³)**: High concentration areas.\n  - **Orange (60 to 80 µg/m³)**: Very high concentration areas.\n  - **Dark Orange/Red (80 to 100 µg/m³)**: Extremely high concentration areas.\n\n### Areas with High Pollution:\n- **Northern India (especially the Indo-Gangetic Plain)**: This area has the highest levels of PM2.5, with significant portions in the dark orange/red category, indicating severe pollution levels (80 to 100 µg/m³).\n- **North-Central India**: Also shows high pollution levels, though not as extreme as the northern region, still falling into the moderate to very high concentration range (40 to 80 µg/m³).\n\n### Population and Area Exposure:\n- **Population Exposure**: \n  - Around 25.8% of India's population is exposed to PM2.5 levels in the 20-30 µg/m³ range.\n  - A significant portion, 19.7%, is exposed to 30-40 µg/m³.\n  - A smaller portion (1.2%) is exposed to the highest levels (80-100 µg/m³).\n  \n- **Area Exposure**:\n  - Most of India’s land area (32.2%) falls in the 5-20 µg/m³ range.\n  - Followed by 20.1% of the area in the 20-30 µg/m³ range.\n  - Even smaller portions of land area (2.1%) fall into the highest pollution category (80-100 µg/m³).\n\n### Standards:\n- **India Annual Standard**: 40 µg/m³\n- **WHO Annual Guideline**: 5 µg/m³\n\nIn summary, the map highlights significant air quality concerns, especially in northern India, where pollution levels exceed both national and international guidelines, posing high health risks to the population in these regions.